# SnowExSQL Database 

 
__Tutorial Author Micah'__: [Micah Sandusky](https://github.com/micah-prime)

__Tutorial Author Micah_o__: [Micah Johnson](https://github.com/micahjohnson150)

[SnowEx](https://snow.nasa.gov/campaigns/snowex) has introduced a unique opportunity to study SWE in a way that's unprecedented, but with more data comes new challenges. 

![examples](./images/snowex_database/data_examples.png)


<!-- 
<img src="https://snowexsql.readthedocs.io/en/latest/_images/gallery_overview_example_12_0.png" alt="Grand Mesa Overview" width="1000px"> -->

**The SnowEx database is a resource that shortcuts the time it takes to ask cross dataset questions**

      
- Standardizing diverse data
- Cross referencing data
- Provenance!
- Added GIS functionality
- Connect w/ ArcGIS or QGIS!
- **CITABLE** 

    * [*2022- Estimating snow accumulation and ablation with L-band interferometric synthetic aperture radar (InSAR)*](https://tc.copernicus.org/articles/17/1997/2023/tc-17-1997-2023-discussion.html)
    * [*2024 - Thermal infrared shadow-hiding in GOES-R ABI imagery: snow and forest temperature observations from the SnowEx 2020 Grand Mesa field campaign*](https://tc.copernicus.org/articles/18/2257/2024/)
      
      

## What's in it?

* Snow pits - Density, hardness profiles, grain types + sizes
* Manual snow depths - TONS of depths (Can you say spirals?)
* Snow Micropenetrometer (SMP) profiles - (Subsampled to every 100th)
* Snow depth + SWE rasters from ASO Inc.
* GPR
* Pit site notes
* Camera Derived snow depths
* Snow off DEM from USGS 3DEP 
* And almost all the associated metadata

## Technically, what is it?

* PostgreSQL database
* PostGIS extension
* Supports vector and raster data
* And a host of GIS operations
* AND NOW WITH API!

# THIS TUTORIAL IS BEING UPDATED